# Used Cars Analysis Pipeline - Pandas
## 02. Data Cleaning and Preprocessing

### Objectives:
1. Handle missing values with appropriate strategies
2. Remove or treat outliers
3. Fix data type inconsistencies
4. Standardize categorical values
5. Create cleaned dataset for analysis

In [1]:
import sys
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

project_root = os.path.abspath(os.path.join(os.getcwd(), '../../'))
if project_root not in sys.path:
    sys.path.append(project_root)

from src.utils.pandas_utils import (
    load_dataset,
    print_dataset_summary,
    get_dataset_info,
    convert_to_datetime,
    save_dataframe,
    load_config
)

warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

print("✅ Setup complete")

✅ Setup complete


In [2]:
# Load dataset
config_path = os.path.join(project_root, 'config', 'pipeline_config.yaml')
config = load_config(config_path)

full_raw_path = os.path.join(project_root, 'data', 'raw')
dataset_filename = config['datasets']['superstore']['filename']
full_file_path = os.path.join(full_raw_path, dataset_filename)

print(f"Loading dataset: {full_file_path}")
df = pd.read_csv(full_file_path, encoding='ISO-8859-1')

print(f"\n📊 Original Dataset Shape: {df.shape}")
print_dataset_summary(df, "Before Cleaning")

Loading dataset: C:\Users\Administrator\Documents\Luxdev\used-cars-analysis-pipeline\data\raw\superstore_final_dataset.csv

📊 Original Dataset Shape: (9800, 18)

BEFORE CLEANING SUMMARY

📊 Dimensions:
   • Rows: 9,800
   • Columns: 18
   • Memory: 9.93 MB

🔢 Data Types:
   • object: 15 columns
   • float64: 2 columns
   • int64: 1 columns

❌ Missing Data:
   • Total missing cells: 11
   • Missing percentage: 0.01%

🔄 Duplicates:
   • Duplicate rows: 0

📋 Column Categories:
   • Numeric: 3
   • Categorical: 15




In [3]:
# Step 1: Handle Missing Values
print("\n" + "="*70)
print("STEP 1: HANDLING MISSING VALUES")
print("="*70)

missing_summary = df.isnull().sum()
print(f"\nMissing values before cleaning:")
print(missing_summary[missing_summary > 0])

# Strategy: Drop columns with >50% missing, impute others
threshold = 0.5
missing_pct = df.isnull().sum() / len(df)
cols_to_drop = missing_pct[missing_pct > threshold].index.tolist()

if cols_to_drop:
    print(f"\n⚠️  Dropping columns with >{threshold*100}% missing: {cols_to_drop}")
    df = df.drop(columns=cols_to_drop)

# Fill remaining missing values
for col in df.columns:
    if df[col].isnull().any():
        if df[col].dtype in ['float64', 'int64']:
            df[col].fillna(df[col].median(), inplace=True)
        else:
            df[col].fillna(df[col].mode()[0] if not df[col].mode().empty else 'Unknown', inplace=True)

print(f"\n✅ Missing values after cleaning: {df.isnull().sum().sum()}")


STEP 1: HANDLING MISSING VALUES

Missing values before cleaning:
Postal_Code    11
dtype: int64

✅ Missing values after cleaning: 0


In [4]:
# Step 2: Remove Duplicates
print("\n" + "="*70)
print("STEP 2: REMOVING DUPLICATES")
print("="*70)

duplicates_before = df.duplicated().sum()
print(f"\nDuplicate rows before: {duplicates_before:,}")

df = df.drop_duplicates()

duplicates_after = df.duplicated().sum()
print(f"Duplicate rows after: {duplicates_after:,}")
print(f"✅ Removed {duplicates_before - duplicates_after:,} duplicate rows")


STEP 2: REMOVING DUPLICATES

Duplicate rows before: 0
Duplicate rows after: 0
✅ Removed 0 duplicate rows


In [5]:
# Step 3: Convert Date Columns
print("\n" + "="*70)
print("STEP 3: CONVERTING DATE COLUMNS")
print("="*70)

date_cols = [col for col in df.columns if 'date' in col.lower() or 'Date' in col]
print(f"\nDate columns found: {date_cols}")

for col in date_cols:
    df = convert_to_datetime(df, col)

print("\n✅ Date conversions complete")


STEP 3: CONVERTING DATE COLUMNS

Date columns found: ['Order_Date', 'Ship_Date']
Converting 'Order_Date' to datetime...
   ⚠️  Warning: 5841 values became NaT
   ✅ Converted to datetime
Converting 'Ship_Date' to datetime...
   ⚠️  Warning: 5985 values became NaT
   ✅ Converted to datetime

✅ Date conversions complete


In [6]:
# Step 4: Handle Outliers
print("\n" + "="*70)
print("STEP 4: HANDLING OUTLIERS")
print("="*70)

numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
print(f"\nNumeric columns: {numeric_cols}")

# Cap outliers using IQR method
for col in numeric_cols:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    outliers_count = ((df[col] < lower_bound) | (df[col] > upper_bound)).sum()
    
    if outliers_count > 0:
        print(f"\n{col}:")
        print(f"  Outliers detected: {outliers_count} ({outliers_count/len(df)*100:.2f}%)")
        print(f"  Capping to [{lower_bound:.2f}, {upper_bound:.2f}]")
        
        df[col] = df[col].clip(lower=lower_bound, upper=upper_bound)

print("\n✅ Outlier treatment complete")


STEP 4: HANDLING OUTLIERS

Numeric columns: ['Row_ID', 'Postal_Code', 'Sales']

Sales:
  Outliers detected: 1145 (11.68%)
  Capping to [-272.79, 500.64]

✅ Outlier treatment complete


In [7]:
# Step 5: Standardize Categorical Variables
print("\n" + "="*70)
print("STEP 5: STANDARDIZING CATEGORICAL VARIABLES")
print("="*70)

categorical_cols = df.select_dtypes(include=['object']).columns.tolist()
categorical_cols = [c for c in categorical_cols if not any(x in c.lower() for x in ['date', 'id'])]

print(f"\nCategorical columns: {categorical_cols}")

for col in categorical_cols:
    # Strip whitespace
    df[col] = df[col].str.strip()
    
    # Standardize case (title case)
    unique_before = df[col].nunique()
    df[col] = df[col].str.title()
    unique_after = df[col].nunique()
    
    if unique_before != unique_after:
        print(f"  {col}: {unique_before} → {unique_after} unique values")

print("\n✅ Categorical standardization complete")


STEP 5: STANDARDIZING CATEGORICAL VARIABLES

Categorical columns: ['Ship_Mode', 'Customer_Name', 'Segment', 'Country', 'City', 'State', 'Region', 'Category', 'Sub_Category', 'Product_Name']

✅ Categorical standardization complete


In [8]:
# Step 6: Create Cleaned Dataset Summary
print("\n" + "="*70)
print("CLEANING SUMMARY")
print("="*70)

print_dataset_summary(df, "After Cleaning")

# Compare before/after
print("\n📊 Cleaning Impact:")
print(f"  Missing values: 0 (was: {df.isnull().sum().sum()})")
print(f"  Duplicates: 0")
print(f"  Shape: {df.shape}")


CLEANING SUMMARY

AFTER CLEANING SUMMARY

📊 Dimensions:
   • Rows: 9,800
   • Columns: 18
   • Memory: 8.86 MB

🔢 Data Types:
   • object: 13 columns
   • datetime64[ns]: 2 columns
   • float64: 2 columns
   • int64: 1 columns

❌ Missing Data:
   • Total missing cells: 11,826
   • Missing percentage: 6.70%

🔄 Duplicates:
   • Duplicate rows: 0

📋 Column Categories:
   • Numeric: 3
   • Categorical: 13



📊 Cleaning Impact:
  Missing values: 0 (was: 11826)
  Duplicates: 0
  Shape: (9800, 18)


In [9]:
# Step 7: Save Cleaned Dataset
print("\n" + "="*70)
print("SAVING CLEANED DATASET")
print("="*70)

output_dir = os.path.join(project_root, 'data', 'cleaned')
save_dataframe(df, 'superstore_cleaned.csv', output_dir, format='csv')
save_dataframe(df, 'superstore_cleaned.parquet', output_dir, format='parquet')

print("\n✅ Data cleaning pipeline complete!")
print(f"\n📁 Cleaned data saved to: {output_dir}")


SAVING CLEANED DATASET
✅ Saved to: C:\Users\Administrator\Documents\Luxdev\used-cars-analysis-pipeline\data\cleaned\superstore_cleaned.csv
✅ Saved to: C:\Users\Administrator\Documents\Luxdev\used-cars-analysis-pipeline\data\cleaned\superstore_cleaned.parquet

✅ Data cleaning pipeline complete!

📁 Cleaned data saved to: C:\Users\Administrator\Documents\Luxdev\used-cars-analysis-pipeline\data\cleaned
